In [3]:
#imports
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime
import dateutil
from dateutil import parser
from dateutil.parser import parse

In [329]:
#crawling in the vgchartz 
Url ="https://www.vgchartz.com"
Url1 = "https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&showdeveloper=0&showcriticscore=0&showcriticscore=1&showpalsales=0&showreleasedate=0&showreleasedate=1&showuserscore=0&showjapansales=0&showlastupdate=0&showlastupdate=1&showothersales=0&showshipped=0&showshipped=1"
html = requests.get(Url1)
soup = BeautifulSoup(html.content, "html.parser")
Game =list()
Console =list()
Publisher =list()
Critic_Score =list()
Total_Shipped =list()
Release_Date =list()
Last_Update =list()
for i in range(1,101): #need range to 101
    tbl =soup("table")[6]
    for row in tbl("tr"):
        cells = row("td")
        if(len(cells)<7):
            continue
        console_name =cells[3].find('img')
        if(console_name['alt'] == 'All' or console_name['alt'] == 'Series'):
            continue
        Console.append(cells[3].get_text().strip() +console_name['alt'])
        Game.append(cells[2].get_text().strip())
        Publisher.append(cells[4].get_text().strip())
        Critic_Score.append(cells[5].get_text().strip())
        if(cells[6].get_text() == 'N/A'):
            Total_Shipped.append(str(cells[7].get_text().strip()).replace('m','0000').replace('.',''))
        else:
            Total_Shipped.append(str(cells[6].get_text().strip()).replace('m','0000').replace('.',''))
        temp =cells[8].get_text().strip()
        if(temp != 'N/A'):
            Release_Date.append(parse(temp))
        else:
            Release_Date.append(temp)
        Last_Update.append(cells[9].get_text().strip())
    link=soup("table")[7]
    linklist = list()
#for i in range(0,3):
    for row in link("tr"):
        link_cell = row("th")
        for t in link_cell[1].find_all('a'):
            if (t.get_text().strip() != '<<' and t.get_text().strip() != '>>'):
                if(int(t.get_text().strip()) == (i+1)):
                    linklist.append(t)
    next_page = linklist[0]["href"]
    next_page1 = requests.get(Url + next_page)
    soup = BeautifulSoup(next_page1.content, "html.parser")

    
        
        



In [331]:
df=pd.DataFrame({'Game':Game,'Console':Console,'Publisher':Publisher,'Critic_Score':Critic_Score,
                'Total_Shipped':Total_Shipped,'Release_Date':Release_Date,'Last_Update':Last_Update})
df

,Game,Console,Publisher,Critic_Score,Total_Shipped,Release_Date,Last_Update
0,Wii Sports,Wii,Nintendo,7.7,82900000,2006-11-19 00:00:00,N/A
1,Super Mario Bros.,NES,Nintendo,10.0,40240000,1985-10-18 00:00:00,N/A
2,Counter-Strike: Global Offensive,PC,Valve,N/A,40000000,2012-08-21 00:00:00,26th Mar 19
3,Mario Kart 8 Deluxe,NS,Nintendo,9.3,38740000,2017-04-28 00:00:00,19th Nov 18
4,Mario Kart Wii Read the review,Wii,Nintendo,8.2,37380000,2008-04-27 00:00:00,11th Apr 18
...,...,...,...,...,...,...,...
18848,IHRA Drag Racing: Sportsman Edition,XB,Bethesda Softworks,N/A,0020000,2006-06-28 00:00:00,N/A
18849,"Sharin no Kuni, Himawari no Shoujo",X360,5pb,N/A,0020000,2010-10-28 00:00:00,19th Dec 18
18850,Drone Tactics,DS,Atlus,N/A,0020000,2008-05-12 00:00:00,N/A
18851,I Love Horses: Rider's Paradise,Wii,Destineer,N/A,0020000,2010-11-30 00:00:00,N/A


In [332]:
#clean df and saved
clean_data=df.copy()
clean_data["Critic_Score"]=clean_data["Critic_Score"].replace('N/A','0')
clean_data["Last_Update"]=clean_data["Last_Update"].replace('N/A','-')
clean_data["Release_Date"] = clean_data["Release_Date"].replace('N/A','01st Jan 00')
clean_data["Total_Shipped"] =clean_data["Total_Shipped"].replace('N/A','0')
clean_data["Total_Shipped"] = [int(item) for item in clean_data["Total_Shipped"]]
clean_data["Critic_Score"] = [float(item) for item in clean_data["Critic_Score"]]
clean_data.to_csv("GameData.csv")
clean_data
 #save Df

,Game,Console,Publisher,Critic_Score,Total_Shipped,Release_Date,Last_Update
0,Wii Sports,Wii,Nintendo,7.7,82900000,2006-11-19 00:00:00,-
1,Super Mario Bros.,NES,Nintendo,10.0,40240000,1985-10-18 00:00:00,-
2,Counter-Strike: Global Offensive,PC,Valve,0.0,40000000,2012-08-21 00:00:00,26th Mar 19
3,Mario Kart 8 Deluxe,NS,Nintendo,9.3,38740000,2017-04-28 00:00:00,19th Nov 18
4,Mario Kart Wii Read the review,Wii,Nintendo,8.2,37380000,2008-04-27 00:00:00,11th Apr 18
...,...,...,...,...,...,...,...
18848,IHRA Drag Racing: Sportsman Edition,XB,Bethesda Softworks,0.0,20000,2006-06-28 00:00:00,-
18849,"Sharin no Kuni, Himawari no Shoujo",X360,5pb,0.0,20000,2010-10-28 00:00:00,19th Dec 18
18850,Drone Tactics,DS,Atlus,0.0,20000,2008-05-12 00:00:00,-
18851,I Love Horses: Rider's Paradise,Wii,Destineer,0.0,20000,2010-11-30 00:00:00,-
